# Apendice 10.2: Uso de Herramientas (Tool Use)- [Leccion](#leccion)- [Ejercicios](#ejercicios)- [Area de Experimentacion](#area-de-experimentacion)## ConfiguracionEjecuta la siguiente celda de configuracion para cargar tu clave API y establecer la funcion auxiliar.**Nota:** Los prompts de ejemplo se mantienen en ingles ya que las tecnicas de ingenieria de prompts son independientes del idioma y Claude responde de manera mas predecible en ingles.

In [ ]:
!pip install anthropicimport jsonimport anthropic# Correccion de ruta para importar hintsimport sys, osnotebook_dir = os.path.dirname(os.path.abspath("__file__"))if notebook_dir not in sys.path:    sys.path.insert(0, notebook_dir)# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython%store -r API_KEY%store -r MODEL_NAMEclient = anthropic.Anthropic(api_key=API_KEY)

---## LeccionEl uso de herramientas (tool use), tambien conocido como llamada a funciones (function calling), permite a Claude **interactuar con sistemas externos** ejecutando funciones que tu defines. Claude no ejecuta las funciones directamente — en su lugar, el flujo funciona asi:1. **Tu defines las herramientas** como un parametro `tools` en la llamada a la API, usando JSON Schema2. **Claude decide** si necesita una herramienta y genera una solicitud estructurada con el nombre y los argumentos3. **Tu codigo ejecuta** la funcion real y devuelve el resultado a Claude4. **Claude usa el resultado** para formular su respuesta finalLa API moderna de Anthropic soporta el uso de herramientas de forma **nativa** mediante el parametro `tools` en `client.messages.create()`. Esto es mucho mas robusto que los metodos antiguos basados en XML.

La llamada a funciones es util porque expande las capacidades de Claude y le permite manejar tareas mucho mas complejas y de multiples pasos.Algunos ejemplos de herramientas que puedes darle a Claude:- **Calculadora** para operaciones aritmeticas precisas- **Consultas a bases de datos** SQL y recuperacion de datos- **APIs externas** como clima, busqueda, etc.- **Manipulacion de archivos** para leer y escribir datos

### Paso 1: Definir las herramientasLas herramientas se definen como una lista de diccionarios, donde cada uno contiene:- `name`: Nombre de la herramienta (identificador unico)- `description`: Descripcion clara de que hace la herramienta- `input_schema`: Esquema JSON que define los parametros de entradaAqui definimos una herramienta de calculadora:

In [ ]:
# Definir la herramienta calculadora usando JSON Schemacalculator_tool = {    "name": "calculator",    "description": "A calculator that performs basic arithmetic operations. Supports addition, subtraction, multiplication, and division.",    "input_schema": {        "type": "object",        "properties": {            "first_operand": {                "type": "integer",                "description": "First operand (before the operator)"            },            "second_operand": {                "type": "integer",                "description": "Second operand (after the operator)"            },            "operator": {                "type": "string",                "enum": ["+", "-", "*", "/"],                "description": "The arithmetic operation to perform"            }        },        "required": ["first_operand", "second_operand", "operator"]    }}tools = [calculator_tool]print("Herramienta definida:", json.dumps(calculator_tool, indent=2))

### Paso 2: Enviar el mensaje con herramientasAhora enviamos un mensaje a Claude junto con la lista de herramientas disponibles. Claude analizara la pregunta y decidira si necesita usar alguna herramienta.

In [ ]:
# Enviar un mensaje que requiere la calculadoraresponse = client.messages.create(    model=MODEL_NAME,    max_tokens=1024,    tools=tools,    messages=[{"role": "user", "content": "Multiply 1,984,135 by 9,343,116"}])print("Stop reason:", response.stop_reason)print("Content:", response.content)

Observa que:- `stop_reason` es `"tool_use"` — Claude quiere usar una herramienta antes de responder- `response.content` contiene un bloque de tipo `ToolUseBlock` con:  - `name`: El nombre de la herramienta que Claude quiere usar  - `input`: Los argumentos estructurados (un diccionario Python, no XML!)  - `id`: Un identificador unico para esta solicitud de herramienta### Paso 3: Ejecutar la herramienta y devolver el resultadoAhora extraemos la solicitud de herramienta, ejecutamos la funcion real, y devolvemos el resultado a Claude:

In [ ]:
# Funcion calculadora realdef do_pairwise_arithmetic(num1, num2, operation):    if operation == '+':        return num1 + num2    elif operation == "-":        return num1 - num2    elif operation == "*":        return num1 * num2    elif operation == "/":        return num1 / num2    else:        return "Error: Operation not supported."# Extraer el bloque tool_use de la respuestatool_use_block = next(block for block in response.content if block.type == "tool_use")tool_name = tool_use_block.nametool_input = tool_use_block.inputprint(f"Claude quiere usar: {tool_name}")print(f"Con argumentos: {tool_input}")# Ejecutar la funcionresult = do_pairwise_arithmetic(    tool_input["first_operand"],    tool_input["second_operand"],    tool_input["operator"])print(f"Resultado: {result:,}")

Ahora enviamos el resultado de vuelta a Claude. Para esto, construimos una conversacion con:1. El mensaje original del usuario2. La respuesta de Claude (que incluye la solicitud de herramienta)3. Un mensaje `tool_result` con el resultado de la ejecucion

In [ ]:
# Construir la conversacion completa con el resultado de la herramientamessages = [    {"role": "user", "content": "Multiply 1,984,135 by 9,343,116"},    {"role": "assistant", "content": response.content},    {        "role": "user",        "content": [            {                "type": "tool_result",                "tool_use_id": tool_use_block.id,                "content": str(result)            }        ]    }]# Obtener la respuesta final de Claudefinal_response = client.messages.create(    model=MODEL_NAME,    max_tokens=1024,    tools=tools,    messages=messages)print("------------- RESULTADO FINAL -------------")print(final_response.content[0].text)

### El bucle completo de uso de herramientas (Agentic Loop)En la practica, querremos automatizar todo el proceso en un **bucle agentico** que:1. Envia el mensaje a Claude2. Si Claude solicita una herramienta, la ejecuta y devuelve el resultado3. Repite hasta que Claude responda sin solicitar herramientasAqui esta la funcion completa:

In [ ]:
def run_tool_loop(user_message, tools, system_prompt=""):    """Ejecuta el bucle completo de uso de herramientas."""    messages = [{"role": "user", "content": user_message}]        # Diccionario de funciones disponibles    available_functions = {        "calculator": lambda inp: do_pairwise_arithmetic(            inp["first_operand"], inp["second_operand"], inp["operator"]        )    }        while True:        response = client.messages.create(            model=MODEL_NAME,            max_tokens=2048,            system=system_prompt,            tools=tools,            messages=messages        )                # Si Claude no quiere usar herramientas, hemos terminado        if response.stop_reason != "tool_use":            # Extraer el texto de la respuesta final            final_text = next(                (block.text for block in response.content if hasattr(block, "text")),                ""            )            return final_text                # Procesar las solicitudes de herramientas        messages.append({"role": "assistant", "content": response.content})                tool_results = []        for block in response.content:            if block.type == "tool_use":                print(f"  [Herramienta] {block.name}({block.input})")                                # Ejecutar la funcion                func = available_functions.get(block.name)                if func:                    result = func(block.input)                else:                    result = f"Error: Unknown tool '{block.name}'"                                print(f"  [Resultado] {result}")                tool_results.append({                    "type": "tool_result",                    "tool_use_id": block.id,                    "content": str(result)                })                messages.append({"role": "user", "content": tool_results})# Probar el bucleprint("=== Pregunta con herramienta ===")answer = run_tool_loop("What is 1,984,135 multiplied by 9,343,116?", tools)print(f"Respuesta: {answer}")

Que pasa si le hacemos a Claude una pregunta que **no requiere** ninguna herramienta?

In [ ]:
print("=== Pregunta sin herramienta ===")answer = run_tool_loop("Tell me the capital of France.", tools)print(f"Respuesta: {answer}")

Exito! Claude supo no llamar a la herramienta cuando no era necesario. La API moderna maneja esto de forma limpia — si Claude no necesita herramientas, simplemente responde con `stop_reason: "end_turn"` y texto normal.Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---## Ejercicios- [Ejercicio 10.2.1 - SQL](#ejercicio-1021---sql)

### Ejercicio 10.2.1 - SQLEn este ejercicio, definiras herramientas para consultar y escribir en la "base de datos" mas pequena del mundo. Aqui esta la base de datos inicializada, que en realidad es solo un diccionario.

In [ ]:
db = {    "users": [        {"id": 1, "name": "Alice", "email": "alice@example.com"},        {"id": 2, "name": "Bob", "email": "bob@example.com"},        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}    ],    "products": [        {"id": 1, "name": "Widget", "price": 9.99},        {"id": 2, "name": "Gadget", "price": 14.99},        {"id": 3, "name": "Doohickey", "price": 19.99}    ]}

Y aqui esta el codigo de las funciones que escriben y leen de la base de datos.

In [ ]:
def get_user(user_id):    for user in db["users"]:        if user["id"] == user_id:            return user    return Nonedef get_product(product_id):    for product in db["products"]:        if product["id"] == product_id:            return product    return Nonedef add_user(name, email):    user_id = len(db["users"]) + 1    user = {"id": user_id, "name": name, "email": email}    db["users"].append(user)    return userdef add_product(name, price):    product_id = len(db["products"]) + 1    product = {"id": product_id, "name": name, "price": price}    db["products"].append(product)    return product

Para resolver el ejercicio, define una lista de herramientas (`tools_sql`) con las cuatro funciones de la base de datos. Cada herramienta debe incluir:- `name`: El nombre de la funcion- `description`: Que hace la funcion- `input_schema`: Los parametros que acepta, usando JSON SchemaTe proporcionamos una estructura inicial a continuacion.

In [ ]:
# Define tus herramientas aquitools_sql = [    # Agrega las definiciones de herramientas para:    # - get_user (parametro: user_id)    # - get_product (parametro: product_id)    # - add_user (parametros: name, email)    # - add_product (parametros: name, price)]

Cuando estes listo, ejecuta la celda de abajo para probar tus definiciones de herramientas. El codigo ejecuta un bucle agentico completo que procesa la solicitud de Claude y ejecuta las funciones reales.

In [ ]:
# Diccionario de funciones disponiblesavailable_functions = {    "get_user": lambda inp: get_user(inp["user_id"]),    "get_product": lambda inp: get_product(inp["product_id"]),    "add_user": lambda inp: add_user(inp["name"], inp.get("email", f"{inp['name'].lower()}@example.com")),    "add_product": lambda inp: add_product(inp["name"], inp.get("price", 0.0)),}def run_sql_tool_loop(user_message):    """Ejecuta el bucle de herramientas para la base de datos."""    messages = [{"role": "user", "content": user_message}]        while True:        response = client.messages.create(            model=MODEL_NAME,            max_tokens=1024,            tools=tools_sql,            messages=messages        )                if response.stop_reason != "tool_use":            final_text = next(                (block.text for block in response.content if hasattr(block, "text")),                str(response.content)            )            return final_text                messages.append({"role": "assistant", "content": response.content})                tool_results = []        for block in response.content:            if block.type == "tool_use":                func = available_functions.get(block.name)                if func:                    result = func(block.input)                else:                    result = f"Error: Unknown tool '{block.name}'"                                tool_results.append({                    "type": "tool_result",                    "tool_use_id": block.id,                    "content": json.dumps(result) if result else "null"                })                messages.append({"role": "user", "content": tool_results})# Probar con varios ejemplosexamples = [    "Add a user to the database named Deborah.",    "Add a product to the database named Thingo",    "Tell me the name of User 2",    "Tell me the name of Product 3"]for example in examples:    print(f"Pregunta: {example}")    print(f"Respuesta: {run_sql_tool_loop(example)}")    print("=" * 50)

Si lo hiciste correctamente, Claude deberia haber llamado a las funciones `add_user`, `add_product`, `get_user` y `get_product` correctamente, y la base de datos deberia reflejar los cambios.Verifiquemos el estado de la base de datos:

In [ ]:
print("Estado actual de la base de datos:")print(json.dumps(db, indent=2))

Si quieres ver una posible solucion, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### Felicidades!Felicidades por aprender el uso moderno de herramientas con la API nativa de Anthropic! Ve a la ultima seccion del apendice si deseas aprender mas sobre busqueda y RAG.

---## Area de ExperimentacionEsta es un area para que experimentes libremente con el uso de herramientas. Puedes definir nuevas herramientas, modificar las existentes, o probar escenarios mas complejos.

In [ ]:
# Define tus propias herramientas y experimentamy_tools = [    {        "name": "my_tool",        "description": "Describe your tool here",        "input_schema": {            "type": "object",            "properties": {                "param1": {"type": "string", "description": "First parameter"}            },            "required": ["param1"]        }    }]# Prueba tu herramientaresponse = client.messages.create(    model=MODEL_NAME,    max_tokens=1024,    tools=my_tools,    messages=[{"role": "user", "content": "Your prompt here"}])print(response.content)